In [1]:
import numpy as np
import pandas as pd

In [2]:
def load_train_data(path):
    train = pd.read_csv(path)
    
    train['Sex'][train['Sex'] == 'male'] = 0
    train['Sex'][train['Sex'] == 'female'] = 1
    train['Sex'] = train['Sex'].astype(int)
    
    fill_train = train.iloc[:, [0,1,2,4,5,6,7,9]]
    med = fill_train['Age'].median()
    fill_train.loc[:, 'Age'] = fill_train['Age'].fillna(med)
    
    fill_train = fill_train.values
    ids = fill_train[:, 0]
    labels = fill_train[:, 1]
    data = fill_train[:, 2:]
    
    return ids, data, labels, med

In [3]:
train_ids, train_data, train_labels, med = load_train_data('data/train.csv')
print(train_data.shape)

train_mean = np.mean(train_data, axis=0)
train_std = np.std(train_data, axis=0)

train_data = (train_data - train_mean) / train_std

val_data = train_data[-100:]
val_labels = train_labels[-100:]
train_data = train_data[:-100]
train_labels = train_labels[:-100]

(891, 6)


/Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/u

https://www.tensorflow.org/guide/keras?hl=ja

In [4]:
import tensorflow as tf
from tensorflow.keras import layers

In [5]:
class TitanicModel(tf.keras.Model):
    def __init__(self, output_dim):
        super(TitanicModel, self).__init__()
        self.dense1 = layers.Dense(16, activation='relu')
        self.dense2 = layers.Dense(16, activation='relu')
        self.out = layers.Dense(output_dim, activation='sigmoid')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.out(x)
        return x

In [6]:
def scheduler(epoch):
  if epoch < 25:
    return 0.001
  else:
    return 0.001 * 0.9 ** (epoch - 24)

In [7]:
from datetime import datetime

model = TitanicModel(output_dim=1)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.losses.log_loss,
              # loss='binary_crossentropy',
              metrics=['accuracy'])

ckpt_path = './ckpt/titanic/titanic'
logdir="./logs/titanic-" + datetime.now().strftime("%Y%m%d-%H%M%S")
# logdir = "./logs"
callbacks = [tf.keras.callbacks.ModelCheckpoint(ckpt_path, save_best_only=True, monitor='val_acc'),
             tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1),
             tf.keras.callbacks.LearningRateScheduler(scheduler),
            ]
model.fit(train_data, train_labels, batch_size=4, epochs=50, callbacks=callbacks,
          validation_data=(val_data, val_labels))

W0815 18:46:24.159141 140736239809472 module_wrapper.py:136] From /Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.losses.log_loss is deprecated. Please use tf.compat.v1.losses.log_loss instead.

W0815 18:46:25.925914 140736239809472 module_wrapper.py:136] From /Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

W0815 18:46:26.021514 140736239809472 deprecation.py:506] From /Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1633: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0815 18:46:26.171652 14073623980

Train on 791 samples, validate on 100 samples
Epoch 1/50
791/791 [==============================] - 1s 779us/sample - loss: 0.5696 - acc: 0.7257 - val_loss: 0.4578 - val_acc: 0.8000
Epoch 2/50
791/791 [==============================] - 0s 386us/sample - loss: 0.4669 - acc: 0.7876 - val_loss: 0.4053 - val_acc: 0.8200
Epoch 3/50
791/791 [==============================] - 0s 428us/sample - loss: 0.4430 - acc: 0.8066 - val_loss: 0.3912 - val_acc: 0.8400
Epoch 4/50
791/791 [==============================] - 0s 358us/sample - loss: 0.4334 - acc: 0.8091 - val_loss: 0.3779 - val_acc: 0.8300
Epoch 5/50
791/791 [==============================] - 0s 385us/sample - loss: 0.4290 - acc: 0.8180 - val_loss: 0.3716 - val_acc: 0.8500
Epoch 6/50
791/791 [==============================] - 0s 386us/sample - loss: 0.4228 - acc: 0.8167 - val_loss: 0.3594 - val_acc: 0.8500
Epoch 7/50
791/791 [==============================] - 0s 377us/sample - loss: 0.4183 - acc: 0.8180 - val_loss: 0.3543 - val_acc: 0.8500
Ep

In [8]:
cnt = 0
for IDX in range(100):
    predict = model.predict(val_data[IDX:IDX + 1])[0][0]
    if predict > 0.5:
        pred_label = 1
    else:
        pred_label = 0
    # print(pred_label, val_labels[IDX], pred_label == val_labels[IDX])
    if pred_label == val_labels[IDX]:
        cnt += 1
print(cnt)

87


In [9]:
model.load_weights(ckpt_path)

In [10]:
def load_test_data(path, med):
    test = pd.read_csv(path)
    
    test['Sex'][test['Sex'] == 'male'] = 0
    test['Sex'][test['Sex'] == 'female'] = 1
    test['Sex'] = test['Sex'].astype(int)
    
    fill = test.iloc[:, [0,1,3,4,5,6,8]]
    fill.loc[:, 'Age'] = fill['Age'].fillna(med)
    
    fill = fill.values
    ids = fill[:, 0]
    data = fill[:, 1:]
    
    return ids, data

In [11]:
test_ids, test_data = load_test_data('data/test.csv', med)
test_data = (test_data - train_mean) / train_std

/Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/fujino/miniconda3/envs/tf/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/u

In [12]:
predictions = model.predict(test_data)

result = []
for id, pred in zip (test_ids, predictions):
    if pred[0] < 0.5:
        label = 0
    else:
        label = 1
    result.append('%d,%d' % (id, label))
    
with open('result.csv', 'w') as fout:
    fout.write('PassengerId,Survived\n')
    fout.write('\n'.join(result))